In [1]:
from openai import OpenAI

client = OpenAI(api_key="XXX")

In [11]:
assistant = client.beta.assistants.create(
    name="English Speaking Assistant",
    instructions="""
    You are an English-speaking practice assistant. 
    Your goal is to help users improve their English by providing feedback, 
    practicing real-life scenarios, 
    and correcting grammar and pronunciation. 
    Be encouraging and constructive in your responses.
    """,
    tools=[{"type":"file_search"}],
    model="gpt-4o"
)

In [12]:
vector_store = client.beta.vector_stores.create(
    name="conversation logs"
)

In [14]:
file_paths = [
    "내가_가장_가보고_싶은_여행지.txt", 
    "내가_감명_깊게_본_영화나_드라마.txt",
    "스트레스_해소를_위한_나만의_방법.txt"
]

In [15]:
file_streams = [open(path, "rb") for path in file_paths]

In [16]:
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_streams
)

In [17]:
print(file_batch.status)

completed


In [18]:
print(file_batch.file_counts)

FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3)


In [19]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}}
        
)

In [20]:
assistant

Assistant(id='asst_6iIui3CjPVR5GExUoSjgETh6', created_at=1731912347, description=None, instructions='\n    You are an English-speaking practice assistant. \n    Your goal is to help users improve their English by providing feedback, \n    practicing real-life scenarios, \n    and correcting grammar and pronunciation. \n    Be encouraging and constructive in your responses.\n    ', metadata={}, model='gpt-4o', name='English Speaking Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_FBL3nbE8cO2caQi06ZN3jag1'])), top_p=1.0)

In [13]:
vector_store

VectorStore(id='vs_FBL3nbE8cO2caQi06ZN3jag1', created_at=1731912377, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1731912377, metadata={}, name='conversation logs', object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)

In [21]:
thread = client.beta.threads.create()

In [22]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Help me find a document containing a conversation about Norway."
)

In [23]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [24]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)


In [30]:
messages.data[0].content[0].text.value

'I found a document containing a conversation about Norway. It\'s in the file titled "내가_가장_가보고_싶은_여행지.txt" where the tutor expresses a desire to visit Norway, specifically to see the beauty of the fjords【4:0†source】.'

In [5]:
thread

Thread(id='thread_ASpLzQYXDOOZmcaLqaQ8QOjt', created_at=1731912085, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [6]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to practice speaking English. Can you help me?"
)

In [7]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [8]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [10]:
messages.data[0]

Message(id='msg_0evoe0dvNHil4OFT6wGwOpBS', assistant_id='asst_4V0eZNrXjtAhhNrvva07OWD8', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Of course! I'd be happy to help you practice speaking English. We can start with a simple conversation. Let's pretend we're meeting for the first time. You can introduce yourself, and I'll respond. \n\nGo ahead with your introduction!"), type='text')], created_at=1731912184, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_btaRrWgyoDuLetHdoYKqzaoi', status=None, thread_id='thread_ASpLzQYXDOOZmcaLqaQ8QOjt')